In [65]:
import pandas as pd
import numpy as np

## Groupby
- SQL Groupby와 같은 기능
- split -> apply -> combine 과정을 거쳐 연산함

In [2]:
# df.groupby(기준 컬럼)[적용 컬럼].sum()(적용연산)

# data from: 
ipl_data = {'Team': ['Riders', 'Riders', 'Devils', 'Devils', 'Kings',
         'kings', 'Kings', 'Kings', 'Riders', 'Royals', 'Royals', 'Riders'],
         'Rank': [1, 2, 2, 3, 3,4 ,1 ,1,2 , 4,1,2],
         'Year': [2014,2015,2014,2015,2014,2015,2016,2017,2016,2014,2015,2017],
         'Points':[876,789,863,673,741,812,756,788,694,701,804,690]}

df = pd.DataFrame(ipl_data)
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [3]:
# Teams 별로 묶어서 팀별 점수의 합을 추출
df.groupby("Team")['Points'].sum() # type : Series

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

In [4]:
# 두 개 이상의 컬럼으로 groupby 할 수 있음
df.groupby(['Team', 'Year'])['Points'].sum()

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

In [5]:
type(df.groupby(['Team', 'Year'])['Points'].sum())

pandas.core.series.Series

### Hierarchical index
- Groupby 명령의 결과물도 결국 DataFrame
- 두 개의 Column으로 groupby를 할 경우, index가 두 개 생성

In [6]:
h_index = df.groupby(['Team', 'Year'])['Points'].sum()
h_index

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Riders  2014    876
        2015    789
        2016    694
        2017    690
Royals  2014    701
        2015    804
kings   2015    812
Name: Points, dtype: int64

In [7]:
h_index.index

MultiIndex([('Devils', 2014),
            ('Devils', 2015),
            ( 'Kings', 2014),
            ( 'Kings', 2016),
            ( 'Kings', 2017),
            ('Riders', 2014),
            ('Riders', 2015),
            ('Riders', 2016),
            ('Riders', 2017),
            ('Royals', 2014),
            ('Royals', 2015),
            ( 'kings', 2015)],
           names=['Team', 'Year'])

In [8]:
h_index["Devils": "Kings"]

Team    Year
Devils  2014    863
        2015    673
Kings   2014    741
        2016    756
        2017    788
Name: Points, dtype: int64

In [9]:
#Hierarichical index - unstack() : Group으로 묶여진 데이터를 Matrix 형태로 전환해줌
h_index.unstack()

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,NaN,NaN
Kings,741.0,NaN,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,NaN,NaN
kings,NaN,812.0,NaN,NaN


In [10]:
h_index.unstack().fillna(0)

Year,2014,2015,2016,2017
Team,,,,
Devils,863.0,673.0,0.0,0.0
Kings,741.0,0.0,756.0,788.0
Riders,876.0,789.0,694.0,690.0
Royals,701.0,804.0,0.0,0.0
kings,0.0,812.0,0.0,0.0


In [11]:
type(h_index.unstack())

pandas.core.frame.DataFrame

In [12]:
#Hierarchical index - operations : Index level을 변경할 수 있다
h_index.swaplevel()

Year  Team  
2014  Devils    863
2015  Devils    673
2014  Kings     741
2016  Kings     756
2017  Kings     788
2014  Riders    876
2015  Riders    789
2016  Riders    694
2017  Riders    690
2014  Royals    701
2015  Royals    804
      kings     812
Name: Points, dtype: int64

In [13]:
h_index.swaplevel().sortlevel(0) #....? Error? #deprecated...

AttributeError: 'Series' object has no attribute 'sortlevel'

In [14]:
#Hierarchical index - operations : level을 기준으로 연산 수행, level은 column의 순서
h_index.sum(level = 1)

Year
2014    3181
2015    3078
2016    1450
2017    1478
Name: Points, dtype: int64

In [15]:
h_index.sum(level = 0)

Team
Devils    1536
Kings     2285
Riders    3049
Royals    1505
kings      812
Name: Points, dtype: int64

### Groupby - Grouped
- Groupby에 의해 split된 상태를 추출 가능
- 내부에서 Tupel형태로 그룹의 Key : Value 형태로 추출됨

In [16]:
grouped = df.groupby('Team')

type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

In [17]:
for name,group in grouped:
    print(name)
    print(group)

Devils
     Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673
Kings
    Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788
Riders
      Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690
Royals
      Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804
kings
    Team  Rank  Year  Points
5  kings     4  2015     812


In [18]:
for data in grouped:
    print(data)

('Devils',      Team  Rank  Year  Points
2  Devils     2  2014     863
3  Devils     3  2015     673)
('Kings',     Team  Rank  Year  Points
4  Kings     3  2014     741
6  Kings     1  2016     756
7  Kings     1  2017     788)
('Riders',       Team  Rank  Year  Points
0   Riders     1  2014     876
1   Riders     2  2015     789
8   Riders     2  2016     694
11  Riders     2  2017     690)
('Royals',       Team  Rank  Year  Points
9   Royals     4  2014     701
10  Royals     1  2015     804)
('kings',     Team  Rank  Year  Points
5  kings     4  2015     812)


In [19]:
# Grouped의 Group된  data를 가져옴
grouped.get_group("kings")

,Team,Rank,Year,Points
5,kings,4,2015,812


추출된 Group정보에는 세 가지 유형의 apply가 가능함
   - Aggregation : 요약된 통계 정보를 추출
   - Transformation : 해당 정보를 변환해줌
   - Filtration : 특정 정보를 제거하여 보여주는 필터링 기능

### Aggregation

In [20]:
grouped.agg(sum)

,Rank,Year,Points
Team,,,
Devils,5,4029,1536
Kings,5,6047,2285
Riders,7,8062,3049
Royals,5,4029,1505
kings,4,2015,812


In [21]:
grouped.agg(np.mean)

,Rank,Year,Points
Team,,,
Devils,2.500000,2014.500000,768.000000
Kings,1.666667,2015.666667,761.666667
Riders,1.750000,2015.500000,762.250000
Royals,2.500000,2014.500000,752.500000
kings,4.000000,2015.000000,812.000000


In [22]:
# 특정 컬럼에 여러 개의 function을 Apply할 수 있음
grouped['Points'].agg([np.sum, np.mean, np.min, np.max])

,sum,mean,amin,amax
Team,,,,
Devils,1536,768.000000,673,863
Kings,2285,761.666667,741,788
Riders,3049,762.250000,690,876
Royals,1505,752.500000,701,804
kings,812,812.000000,812,812


### Transformation
- Aggregation과 달리 Key값 별로 요약된 정보가 아니라
- 개별 데이터의 변환을 지원

In [23]:
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [24]:
# max, min과 같이 Series에 적용되는 데이터들은 key값을 기준으로
# Grouped된 데이터에 해당 function을 적용시킨 결과를 기준으로 결과를 출력한다.
score = lambda x : (x.max())
grouped.transform(score)

,Rank,Year,Points
0,2,2017,876
1,2,2017,876
2,3,2015,863
3,3,2015,863
4,3,2017,788
5,4,2015,812
6,3,2017,788
7,3,2017,788
8,2,2017,876
9,4,2015,804


In [25]:
score = lambda x : (x -  x.mean()) / x.std()
grouped.transform(score)

,Rank,Year,Points
0,-1.500000,-1.161895,1.284327
1,0.500000,-0.387298,0.302029
2,-0.707107,-0.707107,0.707107
3,0.707107,0.707107,-0.707107
4,1.154701,-1.091089,-0.860862
5,NaN,NaN,NaN
6,-0.577350,0.218218,-0.236043
7,-0.577350,0.872872,1.096905
8,0.500000,0.387298,-0.770596
9,0.707107,-0.707107,-0.707107


### Groupby - filter
- 특정 조건으로 데이터를 검색할 때 사용
- filter 내에는 boolean 조건이 존재해야한다
- len(x)는 grouped된 dataframe 개수

In [26]:
df.groupby('Team').filter(lambda x : len(x) >= 3)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
4,Kings,3,2014,741
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
11,Riders,2,2017,690


In [27]:
df

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


In [28]:
df.groupby("Team").filter(lambda x : df["Points"].sum() > 1200)

,Team,Rank,Year,Points
0,Riders,1,2014,876
1,Riders,2,2015,789
2,Devils,2,2014,863
3,Devils,3,2015,673
4,Kings,3,2014,741
5,kings,4,2015,812
6,Kings,1,2016,756
7,Kings,1,2017,788
8,Riders,2,2016,694
9,Royals,4,2014,701


## Case Practice

In [32]:
import dateutil

df_phone = pd.read_csv('./data/phone_data.csv')
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,15/10/14 06:58,34.429,data,2014-11,data,data
1,1,15/10/14 06:58,13.000,call,2014-11,Vodafone,mobile
2,2,15/10/14 14:46,23.000,call,2014-11,Meteor,mobile
3,3,15/10/14 14:48,4.000,call,2014-11,Tesco,mobile
4,4,15/10/14 17:27,4.000,call,2014-11,Tesco,mobile


In [34]:
df_phone.shape

(830, 7)

In [36]:
df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse, dayfirst=True)
df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [37]:
df_phone.groupby('month')['duration'].sum()

month
2014-11    26639.441
2014-12    14641.870
2015-01    18223.299
2015-02    15522.299
2015-03    22750.441
Name: duration, dtype: float64

In [40]:
df_phone[df_phone['item'] == 'call'].groupby('network')['duration'].sum()

network
Meteor        7200.0
Tesco        13828.0
Three        36464.0
Vodafone     14621.0
landline     18433.0
voicemail     1775.0
Name: duration, dtype: float64

In [41]:
df_phone.groupby(['month', 'item'])['date'].count()

month    item
2014-11  call    107
         data     29
         sms      94
2014-12  call     79
         data     30
         sms      48
2015-01  call     88
         data     31
         sms      86
2015-02  call     67
         data     31
         sms      39
2015-03  call     47
         data     29
         sms      25
Name: date, dtype: int64

In [43]:
# DataFrame으로 보기
df_phone.groupby(['month', 'item'])['network'].count().unstack()

item,call,data,sms
month,,,
2014-11,107,29,94
2014-12,79,30,48
2015-01,88,31,86
2015-02,67,31,39
2015-03,47,29,25


In [46]:
# .agg({column : function}) = 해당 컬럼에 function을 적용해라
# .as_index = False -> 인덱스를 따로 적용하지 않는다.
df_phone.groupby('month', as_index = False).agg({'duration' : 'sum'})

,month,duration
0,2014-11,26639.441
1,2014-12,14641.870
2,2015-01,18223.299
3,2015-02,15522.299
4,2015-03,22750.441


In [47]:
df_phone.groupby('network', as_index = False).agg({'duration' : 'mean'})

,network,duration
0,Meteor,83.137931
1,Tesco,164.773810
2,Three,170.004651
3,Vodafone,68.697674
4,data,34.429000
5,landline,438.880952
6,special,1.000000
7,voicemail,65.740741
8,world,1.000000


In [49]:
df_phone.groupby(['month', 'item']).agg({'duration' : sum,
                                         'network_type' : 'count',
                                         'date' : 'first'})

duration  network_type                date
month   item                                             
2014-11 call  25547.000           107 2014-10-15 06:58:00
        data    998.441            29 2014-10-15 06:58:00
        sms      94.000            94 2014-10-16 22:18:00
2014-12 call  13561.000            79 2014-11-14 17:24:00
        data   1032.870            30 2014-11-13 06:58:00
        sms      48.000            48 2014-11-14 17:28:00
2015-01 call  17070.000            88 2014-12-15 20:03:00
        data   1067.299            31 2014-12-13 06:58:00
        sms      86.000            86 2014-12-15 19:56:00
2015-02 call  14416.000            67 2015-01-15 10:36:00
        data   1067.299            31 2015-01-13 06:58:00
        sms      39.000            39 2015-01-15 12:23:00
2015-03 call  21727.000            47 2015-02-12 20:15:00
        data    998.441            29 2015-02-13 06:58:00
        sms      25.000            25 2015-02-19 18:46:00

In [51]:
df_phone.groupby(['month', 'item']).agg({'duration' : [min, max, sum],
                                       'date': ['first', 'last', 'nunique'],
                                       'network_type' : 'count'})

duration                                      date  \
                  min        max        sum               first   
month   item                                                      
2014-11 call    1.000   1940.000  25547.000 2014-10-15 06:58:00   
        data   34.429     34.429    998.441 2014-10-15 06:58:00   
        sms     1.000      1.000     94.000 2014-10-16 22:18:00   
2014-12 call    2.000   2120.000  13561.000 2014-11-14 17:24:00   
        data   34.429     34.429   1032.870 2014-11-13 06:58:00   
        sms     1.000      1.000     48.000 2014-11-14 17:28:00   
2015-01 call    2.000   1859.000  17070.000 2014-12-15 20:03:00   
        data   34.429     34.429   1067.299 2014-12-13 06:58:00   
        sms     1.000      1.000     86.000 2014-12-15 19:56:00   
2015-02 call    1.000   1863.000  14416.000 2015-01-15 10:36:00   
        data   34.429     34.429   1067.299 2015-01-13 06:58:00   
        sms     1.000      1.000     39.000 2015-01-15 12:23:00   
2015-03 call    2.000  10528.000  21727.000 2015-02-12 20:15:00   
        data   34.429     34.429    998.441 2015-02-13 06:58:00   
        sms     1.000      1.000     25.000 2015-02-19 18:46:00   

                                         network_type  
                            last nunique        count  
month   item                                           
2014-11 call 2014-11-12 19:01:00     104          107  
        data 2014-11-12 06:58:00      29           29  
        sms  2014-11-13 22:31:00      79           94  
2014-12 call 2014-12-14 19:54:00      76           79  
        data 2014-12-12 06:58:00      30           30  
        sms  2014-12-07 23:22:00      41           48  
2015-01 call 2015-01-14 20:47:00      84           88  
        data 2015-01-12 06:58:00      31           31  
        sms  2015-01-14 23:36:00      58           86  
2015-02 call 2015-02-09 17:54:00      67           67  
        data 2015-02-12 06:58:00      31           31  
        sms  2015-02-10 21:40:00      27           39  
2015-03 call 2015-03-04 12:29:00      47           47  
        data 2015-03-13 06:58:00      29           29  
        sms  2015-03-14 00:16:00      17           25

In [60]:
grouped = df_phone.groupby('month').agg({'duration' : [min, max, np.mean]})
grouped.rename(columns = {"min" : "min_duration", 'max':'max_duration', 'mean':'mean_duration'})

duration                           
        min_duration max_duration mean_duration
month                                          
2014-11          1.0       1940.0    115.823657
2014-12          1.0       2120.0     93.260318
2015-01          1.0       1859.0     88.894141
2015-02          1.0       1863.0    113.301453
2015-03          1.0      10528.0    225.251891

In [71]:
grouped = df_phone.groupby('month').agg({'duration' : [min, max, np.mean]})

# 컬럼에서 가장 윗단계 컬럼 제거
grouped.columns = grouped.columns.droplevel(level=0)

# 각 컬럼 이름에 맞는 컬럼으로 rename
grouped.rename(columns = {"min" : "min_duration", 'max':'max_duration', 'mean':'mean_duration'})

,min_duration,max_duration,mean_duration
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


In [72]:
grouped = df_phone.groupby('month').agg({"duration" : [min, max, np.mean]})
grouped

duration                     
             min      max        mean
month                                
2014-11      1.0   1940.0  115.823657
2014-12      1.0   2120.0   93.260318
2015-01      1.0   1859.0   88.894141
2015-02      1.0   1863.0  113.301453
2015-03      1.0  10528.0  225.251891

In [73]:
# 컬럼에서 가장 윗단계 컬럼 제거
grouped.columns = grouped.columns.droplevel(level=0)
grouped

,min,max,mean
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


In [74]:
# 각 column에 접두어 추가
grouped.add_prefix("duration_")

,duration_min,duration_max,duration_mean
month,,,
2014-11,1.0,1940.0,115.823657
2014-12,1.0,2120.0,93.260318
2015-01,1.0,1859.0,88.894141
2015-02,1.0,1863.0,113.301453
2015-03,1.0,10528.0,225.251891


## Pivot table & Crosstab
- index축은 groupby와 동일함
- column에 추가로 labeling 값을 추가하는데 value에 numeric type값을 aggregation하는 형태

### Pivot Table

In [76]:
df_phone = pd.read_csv('./data/phone_data.csv')

df_phone['date'] = df_phone['date'].apply(dateutil.parser.parse,
                                         dayfirst = True)

df_phone.head()

,index,date,duration,item,month,network,network_type
0,0,2014-10-15 06:58:00,34.429,data,2014-11,data,data
1,1,2014-10-15 06:58:00,13.000,call,2014-11,Vodafone,mobile
2,2,2014-10-15 14:46:00,23.000,call,2014-11,Meteor,mobile
3,3,2014-10-15 14:48:00,4.000,call,2014-11,Tesco,mobile
4,4,2014-10-15 17:27:00,4.000,call,2014-11,Tesco,mobile


In [ ]:
#.pivot_table([데이터 컬럼], index = [index로 활용할 column],
#               columns = [컬럼내 unique데이터 별로 컬럼 생성],
#               aggfunc = 적용할 함수,
#               fill_value = NaN 값으로 채울 data)

In [77]:
df_phone.pivot_table(['duration'],
                    index = [df_phone.month, df_phone.item],
                    columns=df_phone.network, aggfunc = 'sum')

duration                                                       \
network        Meteor   Tesco    Three Vodafone      data landline special   
month   item                                                                 
2014-11 call   1521.0  4045.0  12458.0   4316.0       NaN   2906.0     NaN   
        data      NaN     NaN      NaN      NaN   998.441      NaN     NaN   
        sms      10.0     3.0     25.0     55.0       NaN      NaN     1.0   
2014-12 call   2010.0  1819.0   6316.0   1302.0       NaN   1424.0     NaN   
        data      NaN     NaN      NaN      NaN  1032.870      NaN     NaN   
        sms      12.0     1.0     13.0     18.0       NaN      NaN     NaN   
2015-01 call   2207.0  2904.0   6445.0   3626.0       NaN   1603.0     NaN   
        data      NaN     NaN      NaN      NaN  1067.299      NaN     NaN   
        sms      10.0     3.0     33.0     40.0       NaN      NaN     NaN   
2015-02 call   1188.0  4087.0   6279.0   1864.0       NaN    730.0     NaN   
        data      NaN     NaN      NaN      NaN  1067.299      NaN     NaN   
        sms       1.0     2.0     11.0     23.0       NaN      NaN     2.0   
2015-03 call    274.0   973.0   4966.0   3513.0       NaN  11770.0     NaN   
        data      NaN     NaN      NaN      NaN   998.441      NaN     NaN   
        sms       NaN     4.0      5.0     13.0       NaN      NaN     NaN   

                              
network      voicemail world  
month   item                  
2014-11 call     301.0   NaN  
        data       NaN   NaN  
        sms        NaN   NaN  
2014-12 call     690.0   NaN  
        data       NaN   NaN  
        sms        NaN   4.0  
2015-01 call     285.0   NaN  
        data       NaN   NaN  
        sms        NaN   NaN  
2015-02 call     268.0   NaN  
        data       NaN   NaN  
        sms        NaN   NaN  
2015-03 call     231.0   NaN  
        data       NaN   NaN  
        sms        NaN   3.0

In [78]:
# fill_value : NaN값을 채울 data
df_phone.pivot_table(['duration'],
                    index = [df_phone.month, df_phone.item],
                    columns=df_phone.network, aggfunc = 'sum',
                    fill_value = 0)

duration                                                   \
network        Meteor Tesco  Three Vodafone      data landline special   
month   item                                                             
2014-11 call     1521  4045  12458     4316     0.000     2906       0   
        data        0     0      0        0   998.441        0       0   
        sms        10     3     25       55     0.000        0       1   
2014-12 call     2010  1819   6316     1302     0.000     1424       0   
        data        0     0      0        0  1032.870        0       0   
        sms        12     1     13       18     0.000        0       0   
2015-01 call     2207  2904   6445     3626     0.000     1603       0   
        data        0     0      0        0  1067.299        0       0   
        sms        10     3     33       40     0.000        0       0   
2015-02 call     1188  4087   6279     1864     0.000      730       0   
        data        0     0      0        0  1067.299        0       0   
        sms         1     2     11       23     0.000        0       2   
2015-03 call      274   973   4966     3513     0.000    11770       0   
        data        0     0      0        0   998.441        0       0   
        sms         0     4      5       13     0.000        0       0   

                              
network      voicemail world  
month   item                  
2014-11 call       301     0  
        data         0     0  
        sms          0     0  
2014-12 call       690     0  
        data         0     0  
        sms          0     4  
2015-01 call       285     0  
        data         0     0  
        sms          0     0  
2015-02 call       268     0  
        data         0     0  
        sms          0     0  
2015-03 call       231     0  
        data         0     0  
        sms          0     3

###  Crosstab
- 두 칼럼에 교차 빈도, 비율, 덧셈 등을 구할 때 사용
- Pivot Table의 특수한 형태
- User-Item Rating Matrix 등을 만들 때 사용가능

In [79]:
df_movie = pd.read_csv("./data/movie_rating.csv")
df_movie.head()

,critic,title,rating
0,Jack Matthews,Lady in the Water,3.0
1,Jack Matthews,Snakes on a Plane,4.0
2,Jack Matthews,You Me and Dupree,3.5
3,Jack Matthews,Superman Returns,5.0
4,Jack Matthews,The Night Listener,3.0


In [81]:
#pd.crosstab(index = 인덱스로 사용할 컬럼 데이터,
#             columns = 컬럼으로 사용할 컬럼 데이터,
#             values = crosstab내부에 들어갈 데이터,
#             aggfunc = 각 데이터에 적용할 함수)

pd.crosstab(index = df_movie.critic,
           columns = df_movie.title,
           values = df_movie.rating,
           aggfunc = "first").fillna(0)

title,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,You Me and Dupree
critic,,,,,,
Claudia Puig,3.0,0.0,3.5,4.0,4.5,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Jack Matthews,0.0,3.0,4.0,5.0,3.0,3.5
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Toby,0.0,0.0,4.5,4.0,0.0,1.0


In [99]:
# same data with groupby
df_movie.groupby(['critic', 'title'])['rating'].agg('first').unstack().fillna(0)

title,Just My Luck,Lady in the Water,Snakes on a Plane,Superman Returns,The Night Listener,You Me and Dupree
critic,,,,,,
Claudia Puig,3.0,0.0,3.5,4.0,4.5,2.5
Gene Seymour,1.5,3.0,3.5,5.0,3.0,3.5
Jack Matthews,0.0,3.0,4.0,5.0,3.0,3.5
Lisa Rose,3.0,2.5,3.5,3.5,3.0,2.5
Mick LaSalle,2.0,3.0,4.0,3.0,3.0,2.0
Toby,0.0,0.0,4.5,4.0,0.0,1.0


In [101]:
df_movie.groupby(['critic', 'title']).agg({'rating' : 'first'}).unstack().fillna(0)

rating                                      \
title         Just My Luck Lady in the Water Snakes on a Plane   
critic                                                           
Claudia Puig           3.0               0.0               3.5   
Gene Seymour           1.5               3.0               3.5   
Jack Matthews          0.0               3.0               4.0   
Lisa Rose              3.0               2.5               3.5   
Mick LaSalle           2.0               3.0               4.0   
Toby                   0.0               0.0               4.5   

                                                                     
title         Superman Returns The Night Listener You Me and Dupree  
critic                                                               
Claudia Puig               4.0                4.5               2.5  
Gene Seymour               5.0                3.0               3.5  
Jack Matthews              5.0                3.0               3.5  
Lisa Rose                  3.5                3.0               2.5  
Mick LaSalle               3.0                3.0               2.0  
Toby                       4.0                0.0               1.0

## Merge & Concat

### Merge
 - SQL에서 많이사용하는 merge와 같은 기능
 - 특정 컬럼을 기준으로 두 개의 데이터를 하나로 합침
 - .merge(df_a, df_b, on='기준 컬럼')

In [102]:
raw_data = {
        'subject_id': ['1', '2', '3', '4', '5', '7', '8', '9', '10', '11'],
        'test_score': [51, 15, 15, 61, 16, 14, 15, 1, 61, 16]}
df_a = pd.DataFrame(raw_data, columns = ['subject_id', 'test_score'])
df_a

,subject_id,test_score
0,1,51
1,2,15
2,3,15
3,4,61
4,5,16
5,7,14
6,8,15
7,9,1
8,10,61
9,11,16


In [103]:
raw_data = {
        'subject_id': ['4', '5', '6', '7', '8'],
        'first_name': ['Billy', 'Brian', 'Bran', 'Bryce', 'Betty'], 
        'last_name': ['Bonder', 'Black', 'Balwner', 'Brice', 'Btisan']}
df_b = pd.DataFrame(raw_data, columns = ['subject_id', 'first_name', 'last_name'])
df_b

,subject_id,first_name,last_name
0,4,Billy,Bonder
1,5,Brian,Black
2,6,Bran,Balwner
3,7,Bryce,Brice
4,8,Betty,Btisan


In [104]:
pd.merge(df_a, df_b, on='subject_id')
# 만약 공통 컬럼이 아닌 한 쪽에만 존재하는 컬럼을 쓰면 에러

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


In [105]:
# 두 DataFrame이 다른 Column이름을 가지고있을때
pd.merge(df_a, df_b, right_on = 'subject_id', left_on = 'subject_id')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


In [106]:
# left에 위치하는 데이터는 모두 출력하고,
# right는 기준 컬럼을 가지고 join 할 수 있는 데이터만 출력
pd.merge(df_a, df_b, on = 'subject_id', how = 'left')

,subject_id,test_score,first_name,last_name
0,1,51,NaN,NaN
1,2,15,NaN,NaN
2,3,15,NaN,NaN
3,4,61,Billy,Bonder
4,5,16,Brian,Black
5,7,14,Bryce,Brice
6,8,15,Betty,Btisan
7,9,1,NaN,NaN
8,10,61,NaN,NaN
9,11,16,NaN,NaN


In [107]:
# right에 위치하는 데이터는 모두 출력하고,
# left는 기준 컬럼을 가지고 join 할 수 있는 데이터만 출력
pd.merge(df_a, df_b, on = 'subject_id', how = 'right')

,subject_id,test_score,first_name,last_name
0,4,61.0,Billy,Bonder
1,5,16.0,Brian,Black
2,6,NaN,Bran,Balwner
3,7,14.0,Bryce,Brice
4,8,15.0,Betty,Btisan


In [108]:
# 공통된 부분만 출력, default
pd.merge(df_a, df_b, on = 'subject_id', how = 'inner')

,subject_id,test_score,first_name,last_name
0,4,61,Billy,Bonder
1,5,16,Brian,Black
2,7,14,Bryce,Brice
3,8,15,Betty,Btisan


In [111]:
# or 느낌 한 쪽이라도 데이터가 있으면 합쳐서 출력
pd.merge(df_a, df_b, on = 'subject_id', how = 'outer')

,subject_id,test_score,first_name,last_name
0,1,51.0,NaN,NaN
1,2,15.0,NaN,NaN
2,3,15.0,NaN,NaN
3,4,61.0,Billy,Bonder
4,5,16.0,Brian,Black
5,7,14.0,Bryce,Brice
6,8,15.0,Betty,Btisan
7,9,1.0,NaN,NaN
8,10,61.0,NaN,NaN
9,11,16.0,NaN,NaN


In [112]:
# ~_lndex = True : index를 기준으로 merge해라
pd.merge(df_a, df_b, right_index = True, left_index = True)

,subject_id_x,test_score,subject_id_y,first_name,last_name
0,1,51,4,Billy,Bonder
1,2,15,5,Brian,Black
2,3,15,6,Bran,Balwner
3,4,61,7,Bryce,Brice
4,5,16,8,Betty,Btisan


### Concat
 - 같은 형태의 데이터를 붙이는 연산작업
 - .append 와 비슷

In [114]:
df_new = pd.concat([df_a, df_b])
df_new

,subject_id,test_score,first_name,last_name
0,1,51.0,NaN,NaN
1,2,15.0,NaN,NaN
2,3,15.0,NaN,NaN
3,4,61.0,NaN,NaN
4,5,16.0,NaN,NaN
5,7,14.0,NaN,NaN
6,8,15.0,NaN,NaN
7,9,1.0,NaN,NaN
8,10,61.0,NaN,NaN
9,11,16.0,NaN,NaN


In [115]:
df_new.reset_index()

,index,subject_id,test_score,first_name,last_name
0,0,1,51.0,NaN,NaN
1,1,2,15.0,NaN,NaN
2,2,3,15.0,NaN,NaN
3,3,4,61.0,NaN,NaN
4,4,5,16.0,NaN,NaN
5,5,7,14.0,NaN,NaN
6,6,8,15.0,NaN,NaN
7,7,9,1.0,NaN,NaN
8,8,10,61.0,NaN,NaN
9,9,11,16.0,NaN,NaN


In [117]:
df_a.append(df_b).reset_index()

,index,subject_id,test_score,first_name,last_name
0,0,1,51.0,NaN,NaN
1,1,2,15.0,NaN,NaN
2,2,3,15.0,NaN,NaN
3,3,4,61.0,NaN,NaN
4,4,5,16.0,NaN,NaN
5,5,7,14.0,NaN,NaN
6,6,8,15.0,NaN,NaN
7,7,9,1.0,NaN,NaN
8,8,10,61.0,NaN,NaN
9,9,11,16.0,NaN,NaN


In [119]:
# axis인자를 사용해 수평적으로 concat도 가능하다
df_new = pd.concat([df_a, df_b], axis = 1)
df_new

,subject_id,test_score,subject_id,first_name,last_name
0,1,51,4,Billy,Bonder
1,2,15,5,Brian,Black
2,3,15,6,Bran,Balwner
3,4,61,7,Bryce,Brice
4,5,16,8,Betty,Btisan
5,7,14,NaN,NaN,NaN
6,8,15,NaN,NaN,NaN
7,9,1,NaN,NaN,NaN
8,10,61,NaN,NaN,NaN
9,11,16,NaN,NaN,NaN


In [120]:
df_new.reset_index()

,index,subject_id,test_score,subject_id,first_name,last_name
0,0,1,51,4,Billy,Bonder
1,1,2,15,5,Brian,Black
2,2,3,15,6,Bran,Balwner
3,3,4,61,7,Bryce,Brice
4,4,5,16,8,Betty,Btisan
5,5,7,14,NaN,NaN,NaN
6,6,8,15,NaN,NaN,NaN
7,7,9,1,NaN,NaN,NaN
8,8,10,61,NaN,NaN,NaN
9,9,11,16,NaN,NaN,NaN


## Case

In [121]:
import os

In [134]:
files = [file_name for file_name in os.listdir('./data') 
         if file_name.endswith("xlsx")]
files

['sales-feb-2014.xlsx',
 'sales-mar-2014.xlsx',
 'excel-comp-data.xlsx',
 'customer-status.xlsx',
 'sales-jan-2014.xlsx']

In [135]:
files.remove('excel-comp-data.xlsx')

In [138]:
files.sort()
files

['customer-status.xlsx',
 'sales-feb-2014.xlsx',
 'sales-jan-2014.xlsx',
 'sales-mar-2014.xlsx']

In [143]:
df_list = [pd.read_excel('data/' + df_filename) for df_filename in files]
status = df_list[0]
sales = pd.concat(df_list[1:])

In [144]:
status.head()

,account number,name,status
0,740150,Barton LLC,gold
1,714466,Trantow-Barrows,silver
2,218895,Kulas Inc,bronze
3,307599,"Kassulke, Ondricka and Metz",bronze
4,412290,Jerde-Hilpert,bronze


In [145]:
sales.head()

,account number,name,sku,quantity,unit price,ext price,date
0,383080,Will LLC,B1-20000,7,33.69,235.83,2014-02-01 09:04:59
1,412290,Jerde-Hilpert,S1-27722,11,21.12,232.32,2014-02-01 11:51:46
2,412290,Jerde-Hilpert,B1-86481,3,35.99,107.97,2014-02-01 17:24:32
3,412290,Jerde-Hilpert,B1-20000,23,78.90,1814.70,2014-02-01 19:56:48
4,672390,Kuhn-Gusikowski,S1-06532,48,55.82,2679.36,2014-02-02 03:45:20


In [147]:
merge_df = pd.merge(status, sales, on = 'account number', how = 'inner')
merge_df.head()

,account number,name_x,status,name_y,sku,quantity,unit price,ext price,date
0,740150,Barton LLC,gold,Barton LLC,S1-93683,21,10.34,217.14,2014-02-07 08:34:50
1,740150,Barton LLC,gold,Barton LLC,S2-10342,47,96.68,4543.96,2014-02-12 03:36:52
2,740150,Barton LLC,gold,Barton LLC,B1-38851,3,79.49,238.47,2014-02-12 05:09:16
3,740150,Barton LLC,gold,Barton LLC,B1-20000,28,81.39,2278.92,2014-02-15 07:45:16
4,740150,Barton LLC,gold,Barton LLC,B1-38851,17,81.22,1380.74,2014-02-17 17:12:16


In [150]:
merge_df.groupby(['status', 'name_x'])['quantity', 'ext price'].sum().reset_index().sort_values( by = ['status', 'quantity'], ascending = Tru
                                                                                                e)

<ipython-input-150-1a03d40d77a5>:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  merge_df.groupby(['status', 'name_x'])['quantity', 'ext price'].sum().reset_index()


,status,name_x,quantity,ext price
0,bronze,Jerde-Hilpert,581,30617.51
1,bronze,"Kassulke, Ondricka and Metz",366,20543.71
2,bronze,Kulas Inc,738,53507.36
3,bronze,Pollich LLC,219,9298.02
4,bronze,Sanford and Sons,403,24003.93
5,gold,Barton LLC,346,21909.13
6,gold,"Cronin, Oberbrunner and Spencer",543,26809.63
7,gold,Herman LLC,417,22940.82
8,gold,Stokes LLC,449,25680.43
9,silver,"Frami, Hills and Schmidt",338,19634.31


## DB Persistence